In [ ]:
# example: male v female cauacasian
mask_A = (X_test['SEX'] == 1) & (X_test['RAC1P'] == 1)
mask_B = (X_test['SEX'] == 2) & (X_test['RAC1P'] == 1)

shap_male = shap_values.values[mask_A.values]
shap_female = shap_values.values[mask_B.values]
#feature_names =[]
feature_names = all_feature_names
t_test_results = []

print(len(feature_names))

normality_warnings = 0
varience_warnings = 0

for i, feature in enumerate(feature_names):
    group1 = shap_male[:, i]
    group2 = shap_female[:, i]
    
    # Shapiro-Wilk test
    if shapiro(group1).pvalue < 0.05:
        #print("Warining: group 1 is not normally distributed!")
        normality_warnings = normality_warnings+1
    if shapiro(group2).pvalue < 0.05:
        #print("Warining: group 2 is not normally distributed!")
        normality_warnings = normality_warnings+1
    # Levenes test ---> depends on assumption of normality!
    w_stats, p_value = levene(group1, group2, center='mean') 
    if p_value < 0.05:
        #print("Warning: the varience is unequally distributed!")
        varience_warnings = varience_warnings + 1
    # outlier test ---> depends on assumption of normality!
    z1 = np.abs(stats.zscore(group1))
    z2 = np.abs(stats.zscore(group2))

print(feature_names)


import matplotlib.pyplot as plt
import numpy as np

for i, feature in enumerate(feature_names):
    group1 = shap_male[:, i]
    group2 = shap_female[:, i]

    plt.hist(group1, bins=30, color='skyblue', edgecolor='black')

    plt.show()
    plt.hist(group2, bins=30, color='skyblue', edgecolor='black')

    plt.show()
    break